In [90]:
import numpy as np
import transformers
transformers.logging.set_verbosity_error()
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict, Dataset, load_metric
from sklearn.metrics import confusion_matrix
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [68]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

def read_and_tokenize_data(filename):
    d = load_dataset('pandas', data_files=filename, split='train')
    d.remove_columns_(column_names = ['__index_level_0__'])
    d = d.map(tokenize_function, batched=True)
    return d

#### Reading data

In [69]:
train = read_and_tokenize_data("data/authors_small_train.pkl")
validation = read_and_tokenize_data("data/authors_small_validation.pkl")
test = read_and_tokenize_data("data/authors_small_test.pkl")

Using custom data configuration default-04da7e7305fe5469
Reusing dataset pandas (/Users/mgaulia/.cache/huggingface/datasets/pandas/default-04da7e7305fe5469/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade)


  0%|          | 0/1 [00:00<?, ?ba/s]

Using custom data configuration default-59e1e1226ac57182
Reusing dataset pandas (/Users/mgaulia/.cache/huggingface/datasets/pandas/default-59e1e1226ac57182/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade)


  0%|          | 0/1 [00:00<?, ?ba/s]

Using custom data configuration default-f14f2b3b028ae230
Reusing dataset pandas (/Users/mgaulia/.cache/huggingface/datasets/pandas/default-f14f2b3b028ae230/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade)


  0%|          | 0/1 [00:00<?, ?ba/s]

#### Training model

In [73]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels = 4)

loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /Users/mgaulia/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute"

In [74]:
trainer = Trainer(
    model=model,
    train_dataset=train,
    eval_dataset=validation,
    tokenizer=tokenizer
)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [75]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 90
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 36


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=36, training_loss=1.1007377836439345, metrics={'train_runtime': 875.6179, 'train_samples_per_second': 0.308, 'train_steps_per_second': 0.041, 'total_flos': 71041260625920.0, 'train_loss': 1.1007377836439345, 'epoch': 3.0})

#### Prediction and metrics

In [77]:
predictions = trainer.predict(test)
y_true = test["label"]
y_pred = np.argmax(predictions.predictions, axis=-1)
confusion_matrix(y_true, y_pred)

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 5
  Batch size = 8


array([[0, 1],
       [0, 4]])

In [78]:
metric = load_metric("accuracy")
metric.compute(predictions=y_pred, references=y_true)

{'accuracy': 0.8}

#### Saving model

In [79]:
trainer.save_model("models/author_classification/model")
tokenizer.save_pretrained("models/author_classification/tokenizer")

Saving model checkpoint to models/author_classification/model
Configuration saved in models/author_classification/model/config.json
Model weights saved in models/author_classification/model/pytorch_model.bin
tokenizer config file saved in models/author_classification/model/tokenizer_config.json
Special tokens file saved in models/author_classification/model/special_tokens_map.json
tokenizer config file saved in models/author_classification/tokenizer/tokenizer_config.json
Special tokens file saved in models/author_classification/tokenizer/special_tokens_map.json


('models/author_classification/tokenizer/tokenizer_config.json',
 'models/author_classification/tokenizer/special_tokens_map.json',
 'models/author_classification/tokenizer/vocab.json',
 'models/author_classification/tokenizer/merges.txt',
 'models/author_classification/tokenizer/added_tokens.json')

#### Loading this model

In [88]:
model = AutoModelForSequenceClassification.from_pretrained("models/author_classification/model", num_labels = 4)
tokenizer = AutoTokenizer.from_pretrained("models/author_classification/tokenizer")
trainer = Trainer(model = model)
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [89]:
test_data = read_and_tokenize_data("data/authors_small_test.pkl")
predictions = trainer.predict(test_data)
y_pred = np.argmax(predictions.predictions, axis=-1)
confusion_matrix(test_data["label"], y_pred)

Using custom data configuration default-f14f2b3b028ae230
Reusing dataset pandas (/Users/mgaulia/.cache/huggingface/datasets/pandas/default-f14f2b3b028ae230/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade)
Loading cached processed dataset at /Users/mgaulia/.cache/huggingface/datasets/pandas/default-f14f2b3b028ae230/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade/cache-a4a5103efebaf326.arrow
The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 5
  Batch size = 8


array([[0, 1],
       [0, 4]])